In [1]:
import os
import sys
import time
import glob
import numpy as np
import sqlite3
import pandas as pd
conn = sqlite3.connect('MSD')
RNDSEED=100
conn.execute('CREATE index my_index ON songs(artist_id, artist_name, track_id)')

OperationalError: index my_index already exists

In [2]:
q = "SELECT Count(year) FROM songs"
res = conn.execute(q)
ntracks = res.fetchone()[0]
print 'Found',ntracks,'tracks for which we have year info.'

# get all artists with average year
q = "SELECT artist_id,Avg(year),artist_name FROM songs GROUP BY artist_id"
res = conn.execute(q)
artists = res.fetchall()
print 'Found',len(artists),'artists with at least one song for which we have year.'

# order artist per average year
ordered_artists = sorted(artists,key=lambda x:x[0]) # so its reporducible, first sort by artist id
ordered_artists = sorted(artists,key=lambda x:x[1])
print 'Oldest artist:',ordered_artists[0][2]+'('+str(ordered_artists[0][1])+')'
print 'Most recent artist:',ordered_artists[-1][2]+'('+str(ordered_artists[-1][1])+')'

# set random seed
np.random.seed( hash(RNDSEED) )

# info about split
print '*********************************************************'
print 'We split artists by ordering them according to their'
print 'average track year. For every 10 artists, we keep one at'
print 'random for the test set.'
print '*********************************************************'

# get subset artists
#conn_subtmdb = sqlite3.connect(subset_tmdb)
#res = conn_subtmdb.execute('SELECT DISTINCT artist_id FROM songs')
res = conn.execute('SELECT DISTINCT artist_id FROM songs')
subset_artists = map(lambda x: x[0], res.fetchall())
#conn_subtmdb.close()
print 'Found',len(subset_artists),'distinct subset artists.'

# split between train and test, every 10 artists, put one at random in test
train_artists = set()
test_artists = set()
artists_per_slice = 10
nslices = int( len(ordered_artists) / artists_per_slice )
for k in range(nslices):
    pos1 = k * artists_per_slice
    slice_artists = map(lambda x: x[0], ordered_artists[pos1:pos1+artists_per_slice])
    test_pos = np.random.randint(len(slice_artists))
    for aidx,a in enumerate(slice_artists):
        if aidx == test_pos:
            test_artists.add(a)
        else:
            train_artists.add(a)
print 'Split done, we have',len(test_artists),'test artists and',len(train_artists),'train artists.'

# count test tracks
n_test_tracks = 0
for a in test_artists:
    q = "SELECT Count(track_id) FROM songs WHERE artist_id='"+a+"' "
    res = conn.execute(q)
    n_test_tracks += res.fetchone()[0]
print 'We have',n_test_tracks,'test tracks out of',str(ntracks)+'.'

# write train
train_artists_list = sorted(list(train_artists))
train_artists_df = pd.DataFrame(train_artists_list)
train_artists_df.columns = ['artist_id']
train_artists_df.to_sql('train_artists', conn, if_exists='replace', index = False)
q = 'SELECT * FROM songs WHERE artist_id IN (SELECT artist_id FROM train_artists)'
train_df = pd.read_sql(q, conn)

print 'Training Written'

# write test
test_artists_list = sorted(list(test_artists))
test_artists_df = pd.DataFrame(test_artists_list)
test_artists_df.columns = ['artist_id']
test_artists_df.to_sql('test_artists', conn, if_exists='replace', index = False)
q = 'SELECT * FROM songs WHERE artist_id IN (SELECT artist_id FROM test_artists)'
test_df = pd.read_sql(q, conn)
# close connection
conn.close()


Found 433323 tracks for which we have year info.
Found 26979 artists with at least one song for which we have year.
Oldest artist: The Devotions(1990.0)
Most recent artist: Voodoo Circle(2010.0)
*********************************************************
We split artists by ordering them according to their
average track year. For every 10 artists, we keep one at
random for the test set.
*********************************************************
Found 26979 distinct subset artists.
Split done, we have 2697 test artists and 24273 train artists.
We have 43440 test tracks out of 433323.
Training Written


In [3]:
train_df.shape

(389855, 200)

In [4]:
test_df.shape

(43440, 200)

In [5]:
train_df.head()

,track_id,year,artist_name,artist_id,song_title,song_beats_persecond,song_duration,song_end_fade_in,song_start_fade_out,song_key,...,timbre80,timbre81,timbre82,timbre83,timbre84,timbre85,timbre86,timbre87,timbre88,timbre89
0,62221,2001,The Bristols,AR002UA1187B9A637D,Little Baby,2.028249,137.06404,0.276,124.987,5,...,36.297724,2.587756,0.971701,-26.216825,5.050970,-10.341241,3.550053,-6.363039,6.630155,-3.351419
1,77565,2001,The Bristols,AR002UA1187B9A637D,A Picture Of You,1.668476,128.26077,0.225,122.474,9,...,73.077501,8.748606,-17.406284,-13.099049,-25.012023,-12.232571,7.830893,-2.467830,3.321361,-2.315212
2,92900,2001,The Bristols,AR002UA1187B9A637D,The Beating Of My Heart,1.445382,141.13914,0.235,127.652,6,...,70.326789,12.946360,-10.324368,-24.837769,8.766298,-0.920194,18.765484,4.592097,2.219196,0.340061
3,198684,2001,The Bristols,AR002UA1187B9A637D,Who Does She Think She Is,2.567431,108.66893,0.258,105.233,1,...,50.535168,11.552168,-27.247643,-8.782058,-12.042823,-9.539300,28.618106,8.254348,-0.437426,5.662652
4,239150,2001,The Bristols,AR002UA1187B9A637D,I'll Be Gone,2.591111,152.05832,0.222,145.293,9,...,55.818511,13.416927,-6.578979,-18.549397,-3.278718,-2.350345,16.070168,1.395176,2.735530,0.828039


In [6]:
test_df.head()

,track_id,year,artist_name,artist_id,song_title,song_beats_persecond,song_duration,song_end_fade_in,song_start_fade_out,song_key,...,timbre80,timbre81,timbre82,timbre83,timbre84,timbre85,timbre86,timbre87,timbre88,timbre89
0,170301,1996,The Advent,AR00P3M1187FB5428C,Standers,0.000000,421.92934,2.357,413.153,11,...,-77.139789,21.780790,-16.332244,-10.670080,-19.737725,9.761879,-2.011100,11.385275,13.135756,24.001249
1,52759,1997,The Advent,AR00P3M1187FB5428C,Stassis,2.382997,326.05995,2.316,314.787,1,...,-64.547552,65.494634,19.748922,-8.674837,-5.877981,3.651836,-17.474711,9.304395,12.707028,8.728760
2,63184,1997,The Advent,AR00P3M1187FB5428C,Insight,2.379507,420.67546,0.554,415.173,1,...,-101.415990,23.923364,8.288417,-16.748644,-24.777161,3.081281,17.199984,22.654901,6.909601,6.953738
3,73434,1997,The Advent,AR00P3M1187FB5428C,Pro II,2.382579,393.69098,0.554,387.913,3,...,-97.001878,53.439355,-19.812027,-9.818451,-0.580186,-3.291033,0.479780,12.754248,6.141638,12.917254
4,96347,1997,The Advent,AR00P3M1187FB5428C,Stassis (Part II),2.404174,544.46975,0.572,531.365,9,...,-42.108866,32.532428,-25.774773,-18.909753,19.912292,-15.122070,-9.543510,6.609418,8.248219,3.558548


In [7]:
train_df.to_csv('train_csv', header=True,index=True, encoding = 'iso-8859-1')

In [8]:
test_df.to_csv('test_csv', header=True,index=True, encoding = 'iso-8859-1')

In [10]:
conn = sqlite3.connect('MSD')
train_df.to_sql('train', conn, if_exists='replace', index = False)

In [11]:
test_df.to_sql('test', conn, if_exists='replace', index = False)